<a href="https://colab.research.google.com/github/ChukwuemekaAham/fcc_ml_sms_text_classification-/blob/main/Copy_of_Copy_of_Copy_of_Copy_of_Copy_of_Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics



print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
dftrain = pd.read_csv(train_file_path,sep="\t",header=None)
dftrain.columns = ['label', 'text']
dftrain

In [ ]:
dftest = pd.read_csv(test_file_path,sep="\t",header=None)
dftest.columns = ['label', 'text']
dftest

In [ ]:
data = dftrain.append(dftest, ignore_index=True)
data

In [ ]:

texts = data["text"]
num_max = 1000
lab_en = LabelEncoder()
token = Tokenizer(num_words=num_max)
token.fit_on_texts(texts)

mat_data = token.texts_to_matrix(texts,mode='count')
mat_train = token.texts_to_matrix(dftrain,mode='count')
mat_test = token.texts_to_matrix(dftest,mode='count')

max_len = 100
x_train = token.texts_to_sequences(dftrain["text"])
x_test = token.texts_to_sequences(dftest["text"])
train = sequence.pad_sequences(x_train,maxlen=max_len)
test = sequence.pad_sequences(x_test,maxlen=max_len)

train_label = dftrain["label"]
train_id = lab_en.fit_transform(train_label)
test_label = dftest["label"]
test_id = lab_en.fit_transform(test_label)

In [ ]:
feature_len = 5000
   
model = Sequential()
model.add(Embedding(feature_len, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc', metrics.binary_accuracy])
model.summary()

In [ ]:
model.fit(train,train_id,batch_size=32,epochs=4,verbose=1,validation_split=0.2)

In [ ]:

def predict_message(pred_text):
  inputstr = token.texts_to_sequences([pred_text])
  pred = model.predict(inputstr)[0][0]

  if pred >= 0.5:
    label = "spam"
  else:
    label = "ham"

  prediction = [pred, label] 

  return prediction


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")
print(prediction)
test_predictions()